# The Battle of Neighborhoods (Week 2)

## Find the best neighborhood in Toronto to open a Restaurant

### Loading Data and transforming into pandas data frame

In [1]:
#Loading the required libraries
import numpy as np
import pandas as pd

#Module for web scraping
!pip install bs4
from bs4 import BeautifulSoup
import requests

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

data = requests.get(url).text

soup = BeautifulSoup(data, 'html5lib')

In [3]:
# Transforming the html page data into data frame


table_contents=[]
table = soup.find('table') # find a html table in a webpage

for row in table.findAll('td'):
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print(table_contents)
df = pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Getting the latitude and the longitude coordinates of each neighborhood

In [4]:
# Dowmloading Excel File for Longitudes & latitudes
!wget -q -O 'GeoLocation.csv' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv
print("GeoLocation Data Downloaded")

GeoLocation Data Downloaded


In [5]:
latlng = pd.read_csv('GeoLocation.csv')
latlng = latlng.rename(columns={'Postal Code': 'PostalCode'})
latlng.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
#Mergering to orginal data frame
df = pd.merge(df, latlng, on = 'PostalCode', how='inner')
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Toronto Neighborhoods populations broken down by postal code

In [7]:
#Loading Canada Census 2016 data
df_pop = pd.read_csv('https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Tables/File.cfm?T=1201&SR=1&RPP=9999&PR=0&CMA=0&CSD=0&S=22&O=A&Lang=Eng&OFT=CSV',encoding = 'unicode_escape')

#droping unwanted columns
df_pop = df_pop.drop(columns=['Geographic name','Province or territory','Incompletely enumerated Indian reserves and Indian settlements, 2016','Total private dwellings, 2016','Private dwellings occupied by usual residents, 2016'])

#renaming columns
df_pop = df_pop.rename(columns={'Geographic code':'PostalCode','Population, 2016':'Population_2016'})

df_pop.head()

,PostalCode,Population_2016
0,01,35151728.0
1,A0A,46587.0
2,A0B,19792.0
3,A0C,12587.0
4,A0E,22294.0


In [8]:
#Mergering population dataset to orginal data frame

df = pd.merge(df_pop,df, on='PostalCode', how='right')

#sort by population
df = df.sort_values(by=['Population_2016'], ascending=False)

df.head()

,PostalCode,Population_2016,Borough,Neighborhood,Latitude,Longitude
22,M2N,75897.0,North York,Willowdale South,43.770120,-79.408493
0,M1B,66108.0,Scarborough,"Malvern, Rouge",43.806686,-79.194353
18,M2J,58293.0,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
100,M9V,55959.0,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
14,M1V,54680.0,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",43.815252,-79.284577


### Defining FourSquare Credentials and version

In [9]:
#Loading required  all libraries for exploring data

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

#Library to handle JSON files
import json 

#Library to handle request
import requests

#transform JSON file into a Pandas dataframe
from pandas import json_normalize

#Matplotlib & associated plotting libraries
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#Map rendering library
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [10]:
# Considering only boroughs that contain the word toronto

df_toronto = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
print("Rows, columns: ",df_toronto.shape)
df_toronto.head()

Rows, columns:  (39, 6)


,PostalCode,Population_2016,Borough,Neighborhood,Latitude,Longitude
0,M5V,49195.0,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
1,M6H,44950.0,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
2,M5A,41078.0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6K,40957.0,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
4,M6P,40035.0,West Toronto,"High Park, The Junction South",43.661608,-79.464763


In [11]:
CLIENT_ID = 'Y2RY5JWD5QAYHEYKLSDF2WCLW45QN4Y5HLQEDIYULPRJ520B'
CLIENT_SECRET = 'NHJ3YWLBVRG233W5EMV3PJDOQHE3PYFTDJ2H3IPBM2TMYIBJ'
VERSION = '20180605'

print("Your Credentials: \n CLIENT_ID: "+ CLIENT_ID,"\n CLIENT_SECRET: "+CLIENT_SECRET)

Your Credentials: 
 CLIENT_ID: Y2RY5JWD5QAYHEYKLSDF2WCLW45QN4Y5HLQEDIYULPRJ520B 
 CLIENT_SECRET: NHJ3YWLBVRG233W5EMV3PJDOQHE3PYFTDJ2H3IPBM2TMYIBJ


In [12]:
LIMIT = 100
radius = 500

#Exploring all neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    
    for name, lat, lng in zip(names,latitudes, longitudes):
        print(name)
        
        # Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID,
                                                                                                                                   CLIENT_SECRET,
                                                                                                                                    VERSION,
                                                                                                                                    lat,
                                                                                                                                    lng,
                                                                                                                                    radius,
                                                                                                                                    LIMIT)
        
        # Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #return only relevat information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude',
                             'Venue',
                             'Venue Latitude',
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

### Get all the venues in Toronto

In [13]:
#Get all the venues in Toronto
toronto_venues = getNearbyVenues(names= df_toronto['Neighborhood'],
                                latitudes = df_toronto['Latitude'],
                                longitudes = df_toronto['Longitude'])

CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Dufferin, Dovercourt Village
Regent Park, Harbourfront
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
The Danforth  East
Runnymede, Swansea
Little Portugal, Trinity
India Bazaar, The Beaches West
Christie
The Danforth West, Riverdale
Church and Wellesley
Davisville
The Annex, North Midtown, Yorkville
The Beaches
Studio District
St. James Town, Cabbagetown
Davisville North
Parkdale, Roncesvalles
Forest Hill North & West
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Kensington Market, Chinatown, Grange Park
Roselawn
University of Toronto, Harbord
Lawrence Park
Rosedale
Harbourfront East, Union Station, Toronto Islands
Garden District, Ryerson
North Toronto West
Moore Park, Summerhill East
Berczy Park
Central Bay Street
St. James Town
Richmond, Adelaide, King
Enclave of M5E
First Canadian Place, Underground city
Enclave of M4L
Commerce 

In [14]:
#Lets count the number of venues per each neighborhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,68,68,68,68,68,68
Christie,15,15,15,15,15,15
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37
Davisville North,7,7,7,7,7,7


In [15]:
#find out how many unique categories can be curated from all the returned venues

print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 231 unique categories.


### Analyzing each Neighborhood

In [16]:
#one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="",prefix_sep="")

#add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

#Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
#Examine the new dataframe size
toronto_onehot.shape

(1600, 231)

In [18]:
#Lets group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.017241,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Bay Street,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.014706,0.000000,0.000000,0.000000,0.000000
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
5,Church and Wellesley,0.026667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013333,0.000000,...,0.013333,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
6,"Commerce Court, Victoria Hotel",0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000
7,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.027027,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
8,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000


### Cluster Neighborhoods

In [19]:
# Set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

#run K-Means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

#Check cluster labels generated for each row in the data frame
kmeans.labels_[0:10]

array([2, 2, 2, 2, 2, 2, 2, 2, 0, 2], dtype=int32)

In [20]:
# sorting in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#Placing Top 10 venues of each neighborhood in a new dataframe

num_top_venues = 10

indicators = ['st','nd','rd']

#create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#Create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns =columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Restaurant,Pharmacy,Cheese Shop,Beer Bar,Japanese Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
2,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Bar,Rental Car Location,Plane,Boat or Ferry,Boutique
3,Central Bay Street,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Restaurant,Salad Place,Bubble Tea Shop,Japanese Restaurant,Burger Joint,Spa
4,Christie,Grocery Store,Café,Park,Nightclub,Restaurant,Candy Store,Italian Restaurant,Baby Store,Coffee Shop,Donut Shop


In [21]:
toronto_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,Central Bay Street,0.014706,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.014706,0.0,0.0,0.0,0.0
4,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [22]:
toronto_labels = pd.merge(df,toronto_grouped, on='Neighborhood', how='right')

toronto_labels = toronto_labels[['PostalCode','Population_2016','Borough','Neighborhood','Latitude','Longitude']]
toronto_labels.head()

,PostalCode,Population_2016,Borough,Neighborhood,Latitude,Longitude
0,M5V,49195.0,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420
1,M6H,44950.0,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
2,M5A,41078.0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6K,40957.0,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
4,M6P,40035.0,West Toronto,"High Park, The Junction South",43.661608,-79.464763


In [23]:
#Add the KMeans Labels

toronto_merged = toronto_labels

#Add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

#Merge toronto_grouped data with df to add coordinates for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Population_2016,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5V,49195.0,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,2,Airport Lounge,Airport Service,Airport Terminal,Coffee Shop,Harbor / Marina,Bar,Rental Car Location,Plane,Boat or Ferry,Boutique
1,M6H,44950.0,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,2,Bakery,Pharmacy,Café,Grocery Store,Park,Gym / Fitness Center,Brewery,Bank,Bar,Middle Eastern Restaurant
2,M5A,41078.0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,2,Coffee Shop,Pub,Bakery,Park,Restaurant,Breakfast Spot,Café,Theater,Wine Shop,Farmers Market
3,M6K,40957.0,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,2,Café,Coffee Shop,Breakfast Spot,Gym,Stadium,Burrito Place,Restaurant,Climbing Gym,Performing Arts Venue,Bakery
4,M6P,40035.0,West Toronto,"High Park, The Junction South",43.661608,-79.464763,2,Mexican Restaurant,Thai Restaurant,Café,Fast Food Restaurant,Arts & Crafts Store,Discount Store,Diner,Bar,Speakeasy,Bakery


In [24]:
toronto_merged1 = toronto_merged.loc[toronto_merged['Cluster Labels']==0, toronto_merged.columns[[3,4]+list(range(5, toronto_merged.shape[1]))]]

toronto_merged1.shape

(4, 14)

In [25]:
toronto_merged2 = toronto_merged.loc[toronto_merged['Cluster Labels']==1, toronto_merged.columns[[3,4]+list(range(5, toronto_merged.shape[1]))]]

toronto_merged2.shape

(1, 14)

In [26]:
toronto_merged3 = toronto_merged.loc[toronto_merged['Cluster Labels']==2, toronto_merged.columns[[3,4]+list(range(5, toronto_merged.shape[1]))]]

toronto_merged3.shape

(32, 14)

In [27]:
toronto_merged4 = toronto_merged.loc[toronto_merged['Cluster Labels']==3, toronto_merged.columns[[3,4]+list(range(5, toronto_merged.shape[1]))]]

toronto_merged4.shape

(1, 14)

In [28]:
toronto_merged5 = toronto_merged.loc[toronto_merged['Cluster Labels']==5, toronto_merged.columns[[3,4]+list(range(5, toronto_merged.shape[1]))]]

toronto_merged5.shape

(0, 14)

#### Cluster 3 contains highest cluster density. so this is best location to establish a restaurant

In [33]:
# Finding the coordinates for cluster 3 centroid
cluster_3_coord = toronto_merged3[['Latitude', 'Longitude']]
cluster_3_coord = list(cluster_3_coord.values)
lat =[]
lng =[]

for i in cluster_3_coord:
    lat.append(i[0])
    lng.append(i[1])

Blatitude = sum(lat)/len(lat)
Blongitude = sum(lng)/len(lng)
print(Blatitude)
print(Blongitude)


43.66890736874999
-79.39017273437503


In [36]:
# Intstall opencage to reverse lookup the cooridinates
!pip install opencage
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

key = '9bd98fadef5f439a809d25846a442b4d'
geocoder = OpenCageGeocode(key)

results = geocoder.reverse_geocode(Blatitude, Blongitude)
pprint(results)

/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-OpenCE/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
[{'annotations': {'DMS': {'lat': "43° 40' 8.26464'' N",
                          'lng': "79° 23' 24.67644'' W"},
                  'MGRS': '17TPJ2978136365',
                  'Maidenhead': 'FN03hq30en',
                  'Mercator': {'x': -8837675.289, 'y': 5384841.691},
                  'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?way=99454272#map=17/43.66896/-79.39019',
                          'note_url': 'https://www.openstreetmap.org/note/new#map=17/43.66896/-79.39019&layers=N',
                          'url': 'https://

In [39]:
#Obtain the popupstring of the best location
popstring = df_toronto[df_toronto['PostalCode'].str.contains('M5S')]

def str_join(*args):
    return ''.join(map(str, args))

popstring_new = str_join('The Best Neighbourhood to establish a Restaurant is in: ', popstring['Neighborhood'].values,  ' in ' ,  popstring['Borough'].values)


print(popstring_new)

The Best Neighbourhood to establish a Restaurant is in: ['University of Toronto, Harbord'] in ['Downtown Toronto']


In [40]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geographical coordinate of Toronto are {}, {}.".format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


### Plot the location on the map

In [47]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lng, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
folium.CircleMarker([Blatitude, Blongitude],
                    radius=50,
                    popup='Toronto',
                    color='red',
                    ).add_to(map_clusters)

# Interactive marker
map_clusters.add_child(folium.ClickForMarker(popup=popstring_new))
       
#map_clusters
map_clusters

### Conclusion

In this project, using k-means cluster algorithm seperated the densly populated neighborhoods into clusters and found the best cluster and identified the centriod of that cluster by getting the location coordinates. This shows the best location to establish a new restraunt.